In [ ]:
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
import json
import cv2
from pycocotools.coco import COCO
from PIL import Image
import torch
import os
import matplotlib.pyplot as plt
#import colormap as cmap
import numpy as np


def visualize(index, masks, bounding_boxes):
    image = coco.loadImgs(coco.getImgIds()[index])[0]    
    anno_ids = coco.getAnnIds(imgIds=[image["id"]])
    annos = coco.loadAnns(anno_ids) 
    
    """new_annos = []
    for single in annos:
        single = get_angles(single)
        new_annos.append([single])
    
    annos = new_annos"""
    filename = os.path.join(data_folder,image["file_name"])
    img = cv2.imread(str(filename))
    print(img.shape)
    for anno in annos:
        #print(anno)
        anno = get_angles(anno)
        #c = tuple([int(x) for x in anno["category_id"]])         
        if masks:
            points = np.array(anno["segmentation"], dtype=np.int32).reshape(1,-1,2)
            img = cv2.polylines(img, points, True, 3)
        
        if bounding_boxes:  
                x,y,w,h,a = np.array(anno["bbox"]).astype(np.int32)
                #img = cv2.rectangle(img, (x,y),(x + w,y + h),(0,255,255), 3) 
                img = draw_xywha(img,x,y,w,h,a)
                cat = coco.cats[anno['category_id']]['name']
                cv2.putText(img, cat, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX ,  0.8, 1, cv2.LINE_AA)            
                
    
    plt.imshow(img[...,::-1])
    plt.axis("off")
    plt.grid(False)
    plt.tight_layout()

def get_angles(ann):
    
    #print(ann['bbox'])
    img_id = ann['image_id']
    new_ann = None
    # get width and height
    if not 'rbbox' in ann:
        # using COCO dataset. 4 = [x1,y1,w,h]
        coco = True
        # convert COCO format: x1,y1,w,h to x,y,w,h
        ann['bbox'][0] = ann['bbox'][0] + ann['bbox'][2] / 2
        ann['bbox'][1] = ann['bbox'][1] + ann['bbox'][3] / 2
        ann['bbox'].append(0)
        if ann['bbox'][2] > ann['bbox'][3]:
            ann['bbox'][2],ann['bbox'][3] = ann['bbox'][3],ann['bbox'][2]
            ann['bbox'][4] -= 90
        new_ann = ann['bbox']
    else:
        # using rotated bounding box datasets. 5 = [cx,cy,w,h,angle]
        assert len(ann['rbbox']) == 5, 'Unknown bbox format' # x,y,w,h,a
        new_ann = ann['rbbox']
        

        if new_ann[2] > new_ann[3]:
            new_ann[2], new_ann[3] = new_ann[3],new_ann[2]
            new_ann[4] -= 90 if new_ann[4] > 0 else -90

    if new_ann[2] == new_ann[3]:
        new_ann[3] += 1 # force that w < h
        
    new_ann[4] = np.clip(new_ann[4], -90.0, 90.0 - 1e-14)

    assert new_ann[2] < new_ann[3]
    assert new_ann[4] >= -90 and new_ann[4] < 90

    # override original bounding box with rotated bounding box
    ann['bbox'] = torch.Tensor(new_ann)
    #print(ann['bbox'])
    return ann

    
def draw_xywha(im, x, y, w, h, angle, color=(255,0,0), linewidth=5):
    '''
    im: image numpy array, shape(h,w,3), RGB
    angle: degree
    '''
    c, s = np.cos(angle/180*np.pi), np.sin(angle/180*np.pi)
    R = np.asarray([[c, s], [-s, c]])
    pts = np.asarray([[-w/2, -h/2], [w/2, -h/2], [w/2, h/2], [-w/2, h/2]])
    rot_pts = []
    for pt in pts:
        rot_pts.append(([x, y] + pt @ R).astype(int))
    contours = np.array([rot_pts[0], rot_pts[1], rot_pts[2], rot_pts[3]])
    cv2.polylines(im, [contours], isClosed=True, color=color,
                thickness=linewidth, lineType=cv2.LINE_4)
    return im
    
    
#data_folder = "/localdata/saurabh/dataset/FES/JPEGImages/"
#coco = COCO("/localdata/saurabh/dataset/FES/coco/annotations/instances_default_with_rbbox.json")
data_folder = "/localdata/saurabh/dataset/omnidetector-Flat_(Record_01)/JPEGImages/"
coco = COCO("/localdata/saurabh/dataset/omnidetector-Flat_(Record_01)/coco/annotations/instances_training.json")
plt.figure(figsize=(100,100))
total = len(coco.getImgIds()) - 1

interact(visualize, index=widgets.IntSlider(min=0, max=total, step=1, continuous_update=False), masks=False, bounding_boxes=True);  
plt.show()

# Make the data files

In [1]:
import glob
from collections import defaultdict
import os
import json
import numpy as np
import cv2
from PIL import Image
import torch
###Input####
mode_in = ''
dataset = ''
image_paths = glob.glob(f"/localdata/saurabh/yolov3/data/{dataset}/images/{mode_in}/*.jpg")
############


image_ids = []
image_names = []
#print(image_paths[0])
##### Edit this part according to the name of the files ####
for image in image_paths:
    #print(image)
    image_name = os.path.splitext(os.path.basename(image))[0]
    #print(image_name)
    image_names += [image_name]
    image_name = image_name.split('_')[0]
    image_ids += [image_name]
    #break
    
#print(image_ids)

In [2]:
len(image_ids)

0

In [3]:
sorted(image_names)

[]

In [3]:
def load_anns(img_dir,json_path):
        coco = False
        print(f'Loading annotations {json_path} into memory...')
        with open(json_path, 'r') as f:
            json_data = json.load(f)

        for ann in json_data['annotations']:
            img_id = ann['image_id']
            #print(img_id)
            new_ann = None
            # get width and height 
            if not 'rbbox' in ann:
                # using COCO dataset. 4 = [x1,y1,w,h]
                coco = True
                # convert COCO format: x1,y1,w,h to x,y,w,h
                ann['bbox'][0] = ann['bbox'][0] + ann['bbox'][2] / 2
                ann['bbox'][1] = ann['bbox'][1] + ann['bbox'][3] / 2
                ann['bbox'].append(0)
                if ann['bbox'][2] > ann['bbox'][3]:
                    ann['bbox'][2], ann['bbox'][3] = ann['bbox'][3], ann['bbox'][2]
                    ann['bbox'][4] -= 90
                new_ann = ann['bbox']
            else:
                # using rotated bounding box datasets. 5 = [cx,cy,w,h,angle]
                # x,y,w,h,a
                assert len(ann['rbbox']) == 5, 'Unknown bbox format'
                new_ann = ann['rbbox']

                if new_ann[2] > new_ann[3]:
                    new_ann[2], new_ann[3] = new_ann[3], new_ann[2]
                    new_ann[4] += 90 - np.finfo(np.float32).eps

            if new_ann[2] == new_ann[3]:
                new_ann[3] += 1  # force that w < h

            assert new_ann[2] < new_ann[3]
            assert new_ann[4] >= -90 and new_ann[4] < 90

            # override original bounding box with rotated bounding box
            ann['bbox'] = torch.Tensor(new_ann)
            imgid2anns[img_id].append(ann)

        for img in json_data['images']:
            img_id = img['id']
            #print(img_id)
            assert img_id not in imgid2path
            anns = imgid2anns[img_id]
            # if there is crowd gt, skip this image
            if coco and any(ann['iscrowd'] for ann in anns):
                continue

            img_ids.append(img_id)
            imgid2path[img_id] = os.path.join(img_dir, img['file_name'])
            imgid2info[img['id']] = img

        catids = [cat['id'] for cat in json_data['categories']]

In [4]:
coco =  False
img_ids = []
imgid2info = dict()
imgid2path = dict()
imgid2anns = defaultdict(list)
catids = []

if dataset == 'fes':
    load_anns("/localdata/saurabh/dataset/FES/JPEGImages/", "/localdata/saurabh/dataset/FES/coco/annotations/instances_default_with_rbbox.json" )
elif dataset == 'custom':
    load_anns("/localdata/saurabh/dataset/theodore_v3/images/", "/localdata/saurabh/dataset/theodore_v3/coco/annotations/instances.json")

Loading annotations /localdata/saurabh/dataset/FES/coco/annotations/instances_default_with_rbbox.json into memory...


In [6]:
#### Run this to get image_ids in sequence starting with 1 #####
image_ids = []
for i in range(len(imgid2info)):
    file_name = imgid2info[i+1]['file_name'].split('.')[0]
    if file_name in image_names:
        image_ids += [i+1]

In [7]:
image_ids

[201,
 202,
 203,
 204,
 205,
 206,
 207,
 208,
 209,
 210,
 211,
 212,
 213,
 214,
 215,
 216,
 217,
 218,
 219,
 220,
 221,
 222,
 223,
 224,
 225,
 226,
 227,
 228,
 229,
 230,
 231,
 232,
 233,
 234,
 235,
 236,
 237,
 238,
 239,
 240,
 241,
 242,
 243,
 244,
 245,
 246,
 247,
 248,
 249,
 250,
 251,
 252,
 253,
 254,
 255,
 256,
 257,
 258,
 259,
 260,
 261,
 262,
 263,
 264,
 265,
 266,
 267,
 268,
 269,
 270,
 271,
 272,
 273,
 274,
 275,
 276,
 277,
 278,
 279,
 280,
 281,
 282,
 283,
 284,
 285,
 286,
 287,
 288,
 289,
 290,
 291,
 292,
 293,
 294,
 295,
 296,
 297,
 298,
 299,
 300,
 301]

In [8]:
ind_path = [imgid2path[int(ids)] for ids in image_ids]
annotations = [imgid2anns[int(ids)] for ids in image_ids]

In [9]:
image_names

['Record_00893',
 'Record_00888',
 'Record_00855',
 'Record_00826',
 'Record_00880',
 'Record_00886',
 'Record_00805',
 'Record_00803',
 'Record_00809',
 'Record_00812',
 'Record_00890',
 'Record_00858',
 'Record_00804',
 'Record_00879',
 'Record_00857',
 'Record_00881',
 'Record_00872',
 'Record_00852',
 'Record_00848',
 'Record_00802',
 'Record_00824',
 'Record_00842',
 'Record_00868',
 'Record_00818',
 'Record_00806',
 'Record_00836',
 'Record_00819',
 'Record_00816',
 'Record_00827',
 'Record_00887',
 'Record_00839',
 'Record_00837',
 'Record_00845',
 'Record_00873',
 'Record_00835',
 'Record_00877',
 'Record_00821',
 'Record_00831',
 'Record_00882',
 'Record_00823',
 'Record_00899',
 'Record_00889',
 'Record_00892',
 'Record_00900',
 'Record_00830',
 'Record_00870',
 'Record_00894',
 'Record_00844',
 'Record_00807',
 'Record_00897',
 'Record_00878',
 'Record_00859',
 'Record_00856',
 'Record_00832',
 'Record_00815',
 'Record_00884',
 'Record_00874',
 'Record_00863',
 'Record_00850

In [11]:
label_map = {1:0, 2:1, 3:2, 4:3, 5:4, 6:5}

In [12]:
label_map[3]

2

In [13]:
all_images = []
text_names = open(f'/localdata/saurabh/yolov3/data/{dataset}/{mode_in}_paths.txt', "w")
for i, ann in enumerate(annotations):
    file_name = image_names[i]
    #### Append the path list
    text_names.write(f'/localdata/saurabh/yolov3/data/{dataset}/images/{mode_in}/{file_name}.png')
    text_names.write('\n')
    #### Open the label file for each anno
    text_file = open(f'/localdata/saurabh/yolov3/data/{dataset}/labels/{mode_in}/{file_name}.txt', "w") 
    for anno in ann:
        #print(anno['category_id'])
        text_file.write("%i " %label_map[anno['category_id']])
        text_file.writelines(["%f " % item  for item in anno['bbox']])
        text_file.write("\n")
    
    text_file.close()
    #break
text_names.close()

print(file_name)
    


Record_00828


In [ ]:
annotations

In [ ]:
with open('/localdata/saurabh/dataset/omnidetector-Flat_(Record_01)/coco/annotations/captions_training.json', 'r') as f:
            json_data = json.load(f)

In [ ]:
json_data['images']